### Adjoint solver in GDSFactory

A powerful feature of Meep is the adjoint solver, which can be used to optimize the topology of components. Gmeep contains a function that takes in a GDSFactory component and returns a Meep optimization problem, which can then be run with another Gmeep function. 

`get_meep_adjoint_optimizer` is the first of these functions and it is the one that takes in a gdsfactory component. You must also specify the design region (which usually will overlay some part of the given component), the objective function specifying what you are actually optimizing for, and the design variables that you are initializing the design region to. An example is provided below showing more of the set-up.

After calling this function, `run_meep_adjoint_optimizer` is called to run the optimization with Meep. This function is where you specify, among other things, the bounds of optimization and what algorithm you are using for optimization.

In [ ]:
import gdsfactory as gf
import gdsfactory.simulation.gmeep as gm
import meep as mp
import meep.adjoint as mpa
import autograd.numpy as npa
import numpy as np
from autograd import  tensor_jacobian_product
from meep.adjoint import get_conic_radius_from_eta_e
import matplotlib as plt
import tidy3d

SiO2 = mp.Medium(index=1.45)
Si = mp.Medium(index=3.45)

radius = 2
c = gf.components.bend_circular(radius=radius)
c = c.move([-2.5,-2.5])

design_region_width = radius + 1
design_region_height = radius + 1

eta_e = 0.55
minimum_length = 0.1
filter_radius = get_conic_radius_from_eta_e(minimum_length, eta_e)
eta_i = 0.5
eta_d = 1 - eta_e

resolution = 20
design_region_resolution = int(5 * resolution)

Nx = int(design_region_resolution * design_region_width)
Ny = int(design_region_resolution * design_region_height)

pml_size = 1.0
extend_length = 3
waveguide_width = 0.5
Sx = 2 * pml_size + design_region_width + extend_length
Sy = 2 * pml_size + design_region_height + extend_length
cell_size = (Sx, Sy)

# define the design region
design_variables = mp.MaterialGrid(mp.Vector3(Nx, Ny), SiO2, Si, grid_type="U_MEAN")
design_region = mpa.DesignRegion(
    design_variables,
    volume=mp.Volume(
        center=mp.Vector3(),
        size=mp.Vector3(design_region_width, design_region_height, 0),
    ),
)

def mapping(x, eta, beta):
    # filter
    filtered_field = mpa.conic_filter(
        x,
        filter_radius,
        design_region_width,
        design_region_height,
        design_region_resolution,
    )

    # projection
    projected_field = mpa.tanh_projection(filtered_field, beta, eta)

    projected_field = (npa.rot90(projected_field.T, 2) + projected_field) / 2

    # interpolate to actual materials
    return projected_field.flatten()

# Initial guess
n = Nx * Ny  # number of parameters
seed = 240
np.random.seed(seed)
x0 = mapping(
    np.random.rand(n),
    eta_i,
    128,
)

def J(source, output):
    power = npa.abs(output / source) ** 2
    return npa.mean(power)

# Use function to get optimization problem
opt = gm.get_meep_adjoint_optimizer(
    c,
    J,
    [design_region],
    [design_variables],
    x0,
    resolution=resolution,
    cell_size=(
        Sx,
        Sy,
    ),
    tpml=1.0,
    extend_ports_length=extend_length,
    port_margin=1,
    port_source_offset=1,
    port_monitor_offset=1,
    wavelength_points=10,
)

evaluation_history = []
cur_iter = [0]

# Objective function
def f(v, gradient, cur_beta):
    print(f"Current iteration: {cur_iter[0] + 1}")

    f0, dJ_du = opt([mapping(v, eta_i, cur_beta)])

    plt.figure()
    ax = plt.gca()
    opt.plot2D(
        False,
        ax=ax,
        plot_sources_flag=False,
        plot_monitors_flag=False,
        plot_boundaries_flag=False,
    )
    plt.show()

    if gradient.size > 0:
        gradient[:] = tensor_jacobian_product(mapping, 0)(
            v, eta_i, cur_beta, np.sum(dJ_du, axis=1)
        )

    evaluation_history.append(np.max(np.real(f0)))

    cur_iter[0] = cur_iter[0] + 1

    return np.real(f0)

# Define spatial arrays used to generate bit masks
x_g = np.linspace(-design_region_width / 2 + 5, design_region_width / 2 + 5, Nx)
y_g = np.linspace(-design_region_height / 2, design_region_height / 2, Ny)
X_g, Y_g = np.meshgrid(x_g, y_g, sparse=True, indexing="ij")

# Define the core mask
left_wg_mask = (X_g == -design_region_width / 2 + 5) & (np.abs(Y_g) <= waveguide_width + 5)
top_wg_mask = (X_g <= (waveguide_width/2 + design_region_width / 2 + 5)) & (X_g >= waveguide_width/2 - design_region_width / 2 + 5) & (Y_g == design_region_height)

Si_mask = left_wg_mask | top_wg_mask

# Define the cladding mask
border_mask = (
    (X_g == -design_region_width / 2)
    | (X_g == design_region_width / 2)
    | (Y_g == -design_region_height / 2)
    | (Y_g == design_region_height / 2)
)
SiO2_mask = border_mask.copy()
SiO2_mask[Si_mask] = False

# Set masks for edges
x = np.ones((n,)) * 0.5
x[Si_mask.flatten()] = 1  # set the edges of waveguides to silicon
x[SiO2_mask.flatten()] = 0  # set the other edges to SiO2

# lower and upper bounds
lb = np.zeros((Nx * Ny,))
lb[Si_mask.flatten()] = 1
ub = np.ones((Nx * Ny,))
ub[SiO2_mask.flatten()] = 0

cur_beta = 4
beta_scale = 2
num_betas = 7
update_factor = 12
run_optimization = True

if run_optimization:
    for iters in range(num_betas):
        print("current beta: ", cur_beta)

        if iters != num_betas - 1:
            x[:] = gm.run_meep_adjoint_optimizer(
                n,
                lambda a, g: f(a, g, cur_beta),
                x,
                lower_bound=lb,
                upper_bound=ub,
                maxeval=update_factor,
            )
        else:
            optimized_component = gm.run_meep_adjoint_optimizer(
                n,
                lambda a, g: f(a, g, cur_beta),
                x,
                lower_bound=lb,
                upper_bound=ub,
                maxeval=update_factor,
                get_optimized_component=True,
                opt=opt,
                threshold_offset_from_max=0.09,
            )
        cur_beta = cur_beta * beta_scale

    optimized_component.plot()
    final_figure_of_merit = 10 * np.log10(
        0.5 * np.array(evaluation_history[-1])
    )